# Supervised Learning Technique 

This notebook show how Supervised Machine Learning techniques can be used to predict the Loan defaulters

Algorithms used:<br>
Logistic regression <br>
Random Forest <br>
Extreme Gradient Boost <br>

Also I have used SMOTE and Random OverSampler to balance the class.

### Load the data

In [68]:
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
from AutoClean import AutoClean
import pandas as pd

import sys
import warnings
warnings.filterwarnings("ignore")

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

In [4]:
dataset = pd.read_csv('application_train.csv').iloc[:10000,:]
pipeline = AutoClean(dataset, target='TARGET')
pipeline.save_processed_dataframe('processed_data.csv')

AutoClean process completed in 70.75688 seconds
Logfile saved to: /Users/chanipornnerunchorn/Documents/auto_feature_eng/autoclean.log
Processed dataframe saved to processed_data.csv


In [5]:
application_train = pipeline.output

## Modeling

### Partitioning

In [6]:
#Determine the number of fraud and valid transactions in the dataset

Fraud = application_train[application_train['TARGET']==1]

Valid = application_train[application_train['TARGET']==0]

outlier_fraction = len(Fraud)/float(len(Valid))

print('outlier_fraction for the whole dataset:')
print(outlier_fraction)

print("Fraud Cases : {}".format(len(Fraud)))

print("Valid Cases : {}".format(len(Valid)))

outlier_fraction for the whole dataset:
0.08401084010840108
Fraud Cases : 775
Valid Cases : 9225


In [7]:
from sklearn.model_selection import train_test_split

#Create independent and Dependent Features
columns = application_train.columns.tolist()
# Filter the columns to remove data we do not want 
columns = [c for c in columns if c not in ["TARGET"]]
# Remove columns that are categorical type
columns = [c for c in columns if application_train[c].dtype != 'object']
# Remove columns that are categorical type
columns = [c for c in columns if application_train[c].dtype != 'object']
# Store the variable we are predicting 
target = "TARGET"
# Define a random state 
state = np.random.RandomState(42)
X = application_train[columns]
y = application_train[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(8000, 165) (8000,)
(2000, 165) (2000,)


### More Feature Selection

using RandomForestClassifier

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [9]:
application_train

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,CODE_GENDER_F,CODE_GENDER_M,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_lab,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompa

In [10]:
X_train.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 8407 to 5177
Data columns (total 165 columns):
 #    Column                                             Non-Null Count  Dtype  
---   ------                                             --------------  -----  
 0    SK_ID_CURR                                         8000 non-null   Int64  
 1    CNT_CHILDREN                                       8000 non-null   Int64  
 2    AMT_INCOME_TOTAL                                   8000 non-null   float64
 3    AMT_CREDIT                                         8000 non-null   float64
 4    AMT_ANNUITY                                        8000 non-null   float64
 5    AMT_GOODS_PRICE                                    8000 non-null   float64
 6    REGION_POPULATION_RELATIVE                         8000 non-null   float64
 7    DAYS_BIRTH                                         8000 non-null   Int64  
 8    DAYS_EMPLOYED                                      8000 non-null   Int64 

In [11]:
y_train.info(verbose=True)

<class 'pandas.core.series.Series'>
Int64Index: 8000 entries, 8407 to 5177
Series name: TARGET
Non-Null Count  Dtype
--------------  -----
8000 non-null   Int64
dtypes: Int64(1)
memory usage: 132.8 KB


In [12]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100, random_state=state))
sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier(random_state=RandomState(MT19937) at 0x289B42940))

In [13]:
sel.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False, False, False, False, False, False,
        True, False, False,  True, False, False, False, False, False,
       False, False,  True,  True,  True, False,  True, False, False,
       False, False, False, False, False, False,  True, False,  True,
        True, False,  True, False, False, False, False, False, False,
       False, False,  True, False,  True,  True, False,  True, False,
       False, False, False, False, False, False, False,  True, False,
        True,  True,  True, False,  True, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False,

In [14]:
selected_feat= X_train.columns[(sel.get_support())].tolist()
len(selected_feat)

38

In [15]:
print(selected_feat)

['SK_ID_CURR', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'CNT_FAM_MEMBERS', 'HOUR_APPR_PROCESS_START', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAREA_MEDI', 'TOTALAREA_MODE', 'OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'NAME_INCOME_TYPE_Working', 'WEEKDAY_APPR_PROCESS_START_THURSDAY', 'CODE_GENDER_F', 'CODE_GENDER_M', 'OCCUPATION_TYPE_lab', 'NAME_FAMILY_STATUS_Married']


In [16]:
app_X_train = X_train.copy() #Taking a copy before dropping 
app_X_test = X_test.copy()

In [17]:
X_train = X_train[selected_feat]
X_test = X_test[selected_feat]

In [18]:
print (X_train.shape, X_test.shape)

(8000, 38) (2000, 38)


------------------------------

## Logistic Regression

In [19]:

from sklearn.linear_model import LogisticRegression

logistic_regressor = LogisticRegression(C = 2)

In [20]:
logistic_regressor.fit(X_train,y_train)


LogisticRegression(C=2)

In [21]:
y_pred = logistic_regressor.predict(X_test)

In [22]:
from sklearn.metrics import classification_report,accuracy_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix

n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Logistic Regression errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Logistic Regression errors: 149
Accuracy Score :
0.9255
Confusion matrix :
[[1851    0]
 [ 149    0]]
Classification Report :
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96      1851
         1.0       0.00      0.00      0.00       149

    accuracy                           0.93      2000
   macro avg       0.46      0.50      0.48      2000
weighted avg       0.86      0.93      0.89      2000

ROC AUC score is:  0.5


## Random Forest - Bagging ensemble of Decision trees

In [23]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators = 100, random_state = state, verbose = 1, n_jobs = -1)

In [24]:
random_forest.fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished


RandomForestClassifier(n_jobs=-1,
                       random_state=RandomState(MT19937) at 0x289B42940,
                       verbose=1)

In [25]:
y_pred = random_forest.predict(X_test) 

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


In [26]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Random Forest errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Random Forest errors: 150
Accuracy Score :
0.925
Confusion matrix :
[[1850    1]
 [ 149    0]]
Classification Report :
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96      1851
         1.0       0.00      0.00      0.00       149

    accuracy                           0.93      2000
   macro avg       0.46      0.50      0.48      2000
weighted avg       0.86      0.93      0.89      2000

ROC AUC score is:  0.4997298757428417


## Extreme Gradient Boost Model

In [27]:
from xgboost import XGBClassifier

In [28]:
xgb_classifier = XGBClassifier(n_estimators=100,max_depth=5)

In [29]:
xgb_classifier.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [30]:
y_pred = xgb_classifier.predict(X_test)

In [31]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Extreme Gradient Boost errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Extreme Gradient Boost errors: 158
Accuracy Score :
0.921
Confusion matrix :
[[1829   22]
 [ 136   13]]
Classification Report :
              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96      1851
         1.0       0.37      0.09      0.14       149

    accuracy                           0.92      2000
   macro avg       0.65      0.54      0.55      2000
weighted avg       0.89      0.92      0.90      2000

ROC AUC score is:  0.5376814274163431


# Balancing class - using SMOTETomek

I have used SMOTETomek which is a method of imblearn. SMOTETomek is a hybrid method which uses an under sampling method (Tomek) in with an over sampling method (SMOTE).

In [32]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss

In [33]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 8407 to 5177
Data columns (total 38 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   SK_ID_CURR                           8000 non-null   Int64  
 1   CNT_CHILDREN                         8000 non-null   Int64  
 2   AMT_INCOME_TOTAL                     8000 non-null   float64
 3   AMT_CREDIT                           8000 non-null   float64
 4   AMT_ANNUITY                          8000 non-null   float64
 5   AMT_GOODS_PRICE                      8000 non-null   float64
 6   REGION_POPULATION_RELATIVE           8000 non-null   float64
 7   DAYS_BIRTH                           8000 non-null   Int64  
 8   DAYS_EMPLOYED                        8000 non-null   Int64  
 9   DAYS_REGISTRATION                    8000 non-null   Int64  
 10  DAYS_ID_PUBLISH                      8000 non-null   Int64  
 11  CNT_FAM_MEMBERS            

In [34]:
# cast to float
y_train = y_train.astype(float)

In [35]:
X_train = X_train.astype(float)

In [36]:
y_train.info()

<class 'pandas.core.series.Series'>
Int64Index: 8000 entries, 8407 to 5177
Series name: TARGET
Non-Null Count  Dtype  
--------------  -----  
8000 non-null   float64
dtypes: float64(1)
memory usage: 125.0 KB


In [37]:
# Implementing Oversampling for Handling Imbalanced 
smk = SMOTETomek(random_state=42)
X_res,y_res=smk.fit_resample(X_train,y_train)

In [38]:
X_res.shape,y_res.shape

((14538, 38), (14538,))

In [39]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(y_train)))
print('Resampled dataset shape {}'.format(Counter(y_res)))

Original dataset shape Counter({0.0: 7374, 1.0: 626})
Resampled dataset shape Counter({0.0: 7269, 1.0: 7269})


## Logistic regression

In [40]:
logistic_regressor.fit(X_res,y_res)

LogisticRegression(C=2)

In [41]:
y_pred = logistic_regressor.predict(X_test)

In [42]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Logistic Regression errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Logistic Regression errors: 764
Accuracy Score :
0.618
Confusion matrix :
[[1153  698]
 [  66   83]]
Classification Report :
              precision    recall  f1-score   support

         0.0       0.95      0.62      0.75      1851
         1.0       0.11      0.56      0.18       149

    accuracy                           0.62      2000
   macro avg       0.53      0.59      0.46      2000
weighted avg       0.88      0.62      0.71      2000

ROC AUC score is:  0.5899767584363975


## Random forest

In [43]:
random_forest.fit(X_res,y_res)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.4s finished


RandomForestClassifier(n_jobs=-1,
                       random_state=RandomState(MT19937) at 0x289B42940,
                       verbose=1)

In [44]:
y_pred = random_forest.predict(X_test)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


In [45]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Random Forest errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Random Forest errors: 150
Accuracy Score :
0.925
Confusion matrix :
[[1844    7]
 [ 143    6]]
Classification Report :
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96      1851
         1.0       0.46      0.04      0.07       149

    accuracy                           0.93      2000
   macro avg       0.69      0.52      0.52      2000
weighted avg       0.89      0.93      0.89      2000

ROC AUC score is:  0.5182433583878114


## XGBoost

In [46]:
X_res = pd.DataFrame(data=X_res, columns=selected_feat)

In [47]:
xgb_classifier.fit(X_res,y_res)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [48]:
y_pred = xgb_classifier.predict(X_test)

In [49]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Extreme Gradient Boost errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Extreme Gradient Boost errors: 163
Accuracy Score :
0.9185
Confusion matrix :
[[1826   25]
 [ 138   11]]
Classification Report :
              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96      1851
         1.0       0.31      0.07      0.12       149

    accuracy                           0.92      2000
   macro avg       0.62      0.53      0.54      2000
weighted avg       0.88      0.92      0.89      2000

ROC AUC score is:  0.530159645248895


# Balancing class - RandomOverSampler 

In [50]:
from imblearn.over_sampling import RandomOverSampler

In [51]:
os =  RandomOverSampler(sampling_strategy=1)

In [52]:
X_train_res, y_train_res = os.fit_resample(X_train, y_train)

In [53]:
X_train_res.shape,y_train_res.shape

((14748, 38), (14748,))

In [54]:
print('Original dataset shape {}'.format(Counter(y_train)))
print('Resampled dataset shape {}'.format(Counter(y_train_res)))

Original dataset shape Counter({0.0: 7374, 1.0: 626})
Resampled dataset shape Counter({0.0: 7374, 1.0: 7374})


## Logistic regression

In [55]:
logistic_regressor.fit(X_train_res,y_train_res)

LogisticRegression(C=2)

In [56]:
y_pred = logistic_regressor.predict(X_test)

In [57]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Logistic Regression errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Logistic Regression errors: 756
Accuracy Score :
0.622
Confusion matrix :
[[1160  691]
 [  65   84]]
Classification Report :
              precision    recall  f1-score   support

         0.0       0.95      0.63      0.75      1851
         1.0       0.11      0.56      0.18       149

    accuracy                           0.62      2000
   macro avg       0.53      0.60      0.47      2000
weighted avg       0.88      0.62      0.71      2000

ROC AUC score is:  0.595223332934492


## Random forest

In [58]:
random_forest.fit(X_train_res,y_train_res)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.3s finished


RandomForestClassifier(n_jobs=-1,
                       random_state=RandomState(MT19937) at 0x289B42940,
                       verbose=1)

In [59]:
y_pred = random_forest.predict(X_test)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


In [60]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Random Forest errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Random Forest errors: 156
Accuracy Score :
0.922
Confusion matrix :
[[1843    8]
 [ 148    1]]
Classification Report :
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96      1851
         1.0       0.11      0.01      0.01       149

    accuracy                           0.92      2000
   macro avg       0.52      0.50      0.49      2000
weighted avg       0.86      0.92      0.89      2000

ROC AUC score is:  0.5011947106407203


## XGBoost

In [61]:
X_train_res = pd.DataFrame(data=X_train_res, columns=selected_feat)

In [62]:
X_train_res.shape

(14748, 38)

In [63]:
xgb_classifier.fit(X_train_res,y_train_res)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [64]:
y_pred = xgb_classifier.predict(X_test)

In [65]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Extreme Gradient Boost errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred, target_names=['0','1']))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Extreme Gradient Boost errors: 242
Accuracy Score :
0.879
Confusion matrix :
[[1723  128]
 [ 114   35]]
Classification Report :
              precision    recall  f1-score   support

           0       0.94      0.93      0.93      1851
           1       0.21      0.23      0.22       149

    accuracy                           0.88      2000
   macro avg       0.58      0.58      0.58      2000
weighted avg       0.88      0.88      0.88      2000

ROC AUC score is:  0.5828737595132687
